In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import ast
import os
import json
import matplotlib.pyplot as plt
from nltk import tokenize
import seaborn as sns
import binascii

In [3]:
dataset_pd = pd.read_csv("C:\\Users\\crist\\Documents\\AES tests\\tstnew1_output.csv")

dataset_pd["ENCRYPTED"] = dataset_pd["ENCRYPTED"].apply(lambda t: [i for i in binascii.unhexlify(t)])

def create_character_tokenizer(list_of_strings):
    tokenizer = Tokenizer(filters=None,
                         char_level=True, 
                          split=None,
                         lower=False)
    tokenizer.fit_on_texts(list_of_strings)
    return tokenizer


tokenizer = create_character_tokenizer(dataset_pd["TEXT"])

tokenizer_config = tokenizer.get_config()

word_counts = json.loads(tokenizer_config['word_counts'])
index_word = json.loads(tokenizer_config['index_word'])
word_index = json.loads(tokenizer_config['word_index'])

def strings_to_sequences(tokenizer, list_of_strings):
    sentence_seq = tokenizer.texts_to_sequences(list_of_strings)
    return sentence_seq


seq_texts = strings_to_sequences(tokenizer, dataset_pd["TEXT"])
dataset_pd["TEXT"] = seq_texts


x = dataset_pd["TEXT"]
x = [np.asarray(i) for i in x]

y = dataset_pd["ENCRYPTED"]
y = [np.asarray(i) for i in y]

In [4]:
test_pct = .2
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 50
seq_length = 200
rnn_units = 1024

x_test = x[:int(len(x)*test_pct)]
y_test = y[:int(len(y)*test_pct)]

x_train = x[int(len(x)*test_pct):]
y_train = y[int(len(y)*test_pct):]

tst_full = tf.data.Dataset.from_tensor_slices((x, y))

dataset_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = dataset_train.batch(batch_size, drop_remainder=True)

#dataset_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
#valid_data = dataset_test.batch(batch_size, drop_remainder=True)

train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dense(256))

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath='.\\models\\ckpt',
                                                       save_weights_only=True,
                                                       save_best_only=True)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

history = model.fit(tst_full, epochs=30, callbacks=[checkpoint_callback, tf.keras.callbacks.EarlyStopping(patience=2)])

model.compile(optimizer='sgd', loss='mse')

Epoch 1/30
 18362/250000 [=>............................] - ETA: 6:52 - loss: 5.5341 - sparse_categorical_accuracy: 0.0059

KeyboardInterrupt: 

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [9]:
training_a, testing_a, training_b, testing_b = train_test_split(x, y, test_size = 0.25)
myscaler = StandardScaler()
myscaler.fit(training_a)
training_a = myscaler.transform(training_a)
testing_a = myscaler.transform(testing_a)

In [21]:
training_b = np.asarray(training_b)
training_a = np.asarray(training_a)
testing_b = np.asarray(testing_b)
testing_a = np.asarray(testing_a)

In [32]:
len(training_b[0])

64

In [34]:
m1 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1) #MLPClassifier(hidden_layer_sizes=(9, 12, 10, 15,), activation='relu', solver='adam', max_iter=2000)
m1.fit(training_a, training_b)
predicted_values = m1.predict(testing_a)
print(confusion_matrix(testing_b,predicted_values))
print(classification_report(testing_b,predicted_values))

ValueError: Multioutput target data is not supported with label binarization